# Requirements

Napomena otvori drug Anaconda Prompt i site paketi sto ti trebaat instalirajgi so conda install

1. Vaka otvoram Anaconda Prompt

conda install pytorch torchvision pytorch-cuda=11.8 -c pytorch -c nvidia

2. Napravi check direkno od Anaconda Prompt

python -c "import torch; print(torch.__version__); print(torch.version.cuda); print(torch.cuda.get_device_properties(0)); print(torch.randn(1).cuda()); print(torch.cuda.is_available())"

3. Pa od Anaconda Prompt otvorisi kaj tie notebook filot primer na Disk F mie vo folder VNP/Lab4

(base) C:\Users\Beqir> F:

(base) F:\>cd VNP/Lab4

4. Sega pisi samo "jupyter notebook" i ke ti se otvori

5. Napravi check vo notbook so torch

In [1]:
# Add as many imports as you need.
import torch
import datasets

In [2]:
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.get_device_properties(0))
print(torch.randn(1).cuda())
print(torch.cuda.is_available())

2.5.1
11.8
_CudaDeviceProperties(name='NVIDIA GeForce GTX 1050', major=6, minor=1, total_memory=4095MB, multi_processor_count=5, uuid=c87c7efa-5d3d-89c4-c2b7-218d5d7cd65e, L2_cache_size=0MB)
tensor([-0.4529], device='cuda:0')


True

# Laboratory Exercise - Run Mode (8 points)

## Introduction
This laboratory assignment's primary objective is to fine-tune a pre-trained language model for binary classification on a dataset consisting of Spotify user reviews. The dataset contains two attributes:

+ **review** - A text column containing user feedback, opinions, and experiences with the Spotify application.
+ **sentiment** - A categorical column indicating whether the review has a positive or negative sentiment.

Your task involves training a model to predict the **sentiment** (either "positive" or "negative") based on the content of the **review**.

## The Spotify User Reviews Dataset

Load the dataset using the `datasets` library.

In [3]:
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split

In [4]:
# Write your code here. Add as many boxes as you need.
dataset = load_dataset('csv', data_files='spotify-user-reviews.csv')

In [5]:
dataset['train'].column_names

['review', 'label']

## Dataset Splitting
Partition the dataset into training and testing sets with an 80:20 ratio.


In [6]:
data = pd.DataFrame(dataset['train'])

In [7]:
# Write your code here. Add as many boxes as you need.
train_data, test_data = train_test_split(data, test_size=0.2, stratify=data['label'])

In [8]:
from datasets import Dataset
train_dataset = Dataset.from_dict(train_data)
test_dataset = Dataset.from_dict(test_data)

## Tokenization
Tokenize the texts using the `AutoTokenizer` class.

In [9]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [10]:
def tokenize(batch):
    # Check if 'review' column exists
    return tokenizer(
        batch['review'],
        padding=True,
        truncation=True,
        max_length=128,
    )

In [11]:
# Write your code here. Add as many boxes as you need.

# Apply tokenization
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Ensure the format is set for PyTorch
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [12]:
# Map the string labels to integers (e.g., 'positive' -> 1, 'negative' -> 0)
label_mapping = {"positive": 1, "negative": 0}

def map_labels(example):
    example['label'] = label_mapping[example['label']]
    return example

# Apply the label mapping
train_dataset = train_dataset.map(map_labels)
test_dataset = test_dataset.map(map_labels)


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

## Fine-tuning a Pre-trained Language Model for Classification
Fine-tune a pre-trained language model for classification on the given dataset.

Define the model using the `AutoModelForSequenceClassification` class.

In [13]:
# Write your code here. Add as many boxes as you need.
from transformers import AutoModelForSequenceClassification

# Load pre-trained model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use the first available GPU
    model.to(device)               # Move the model to GPU
else:
    device = torch.device("cpu")   # Use CPU if GPU is not available
    model.to(device)
print(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


cuda


Define the traning parameters using the `TrainingArguments` class.

In [14]:
# Write your code here. Add as many boxes as you need.
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    no_cuda=False,  # Ensure CUDA is used
    logging_dir='./logs',
    fp16=True,
    gradient_accumulation_steps=4,
    seed=42,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

F:\Anaconda\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Define the training using the `Trainer` class.

In [15]:
# Write your code here. Add as many boxes as you need.
from transformers import Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Define metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.207325,0.919000,0.919081,0.918164,0.920000


TrainOutput(global_step=250, training_loss=0.2656896667480469, metrics={'train_runtime': 334.5697, 'train_samples_per_second': 23.911, 'train_steps_per_second': 0.747, 'total_flos': 264934797312000.0, 'train_loss': 0.2656896667480469, 'epoch': 1.0})

Fine-tune (train) the pre-trained lanugage model.

In [ ]:
# Write your code here. Add as many boxes as you need.

Use the trained model to make predictions for the test set.

In [16]:
# Write your code here. Add as many boxes as you need.
predictions = trainer.predict(test_dataset)

Assess the performance of the model by using different metrics provided by the `scikit-learn` library.

In [17]:
# Write your code here. Add as many boxes as you need.
print(predictions.metrics)

{'test_loss': 0.20732472836971283, 'test_accuracy': 0.919, 'test_f1': 0.919080919080919, 'test_precision': 0.9181636726546906, 'test_recall': 0.92, 'test_runtime': 23.5212, 'test_samples_per_second': 85.03, 'test_steps_per_second': 10.629}


# Laboratory Exercise - Bonus Task (+ 2 points)

Implement a machine learning pipeline to classify Spotify user reviews as positive or negative. Use TF-IDF vectorization to transform the review text into numerical features, and train a logistic regression model on the transformed data. Split the dataset into training and testing sets, fit the pipeline on the training data, and evaluate its performance using metrics such as precision, recall, and F1-score. To gain insights into the most influential words or phrases associated with positive and negative reviews, analyze the coefficients from the logistic regression model trained on the TF-IDF features. Present the top keywords for each sentiment in a table or a bar chart to provide a clear understanding of the terms driving user feedback.

In [19]:
# Write your code here. Add as many boxes as you need.
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Vectorize
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_data['review'])
X_test = vectorizer.transform(test_data['review'])

# Labels
y_train = train_data['label']
y_test = test_data['label']


In [21]:
# Train model
clf = LogisticRegression()
clf.fit(X_train, y_train)

# Predict
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.86      0.90      0.88      1000
    positive       0.90      0.85      0.87      1000

    accuracy                           0.88      2000
   macro avg       0.88      0.88      0.88      2000
weighted avg       0.88      0.88      0.88      2000



In [22]:
import numpy as np
import pandas as pd

# Get coefficients
coefficients = clf.coef_.flatten()
top_positive_indices = np.argsort(coefficients)[-10:]  # Top 10 positive
top_negative_indices = np.argsort(coefficients)[:10]   # Top 10 negative

# Get feature names
features = vectorizer.get_feature_names_out()

# Create DataFrame for visualization
top_features = pd.DataFrame({
    "Positive": [features[i] for i in top_positive_indices],
    "Negative": [features[i] for i in top_negative_indices]
})
print(top_features)


    Positive  Negative
0      music       fix
1      enjoy       not
2  excellent  annoying
3    awesome    update
4       nice     worst
5    amazing     doesn
6      great       ads
7       best       why
8       good       but
9       love      only
